# Chapter 5 応用：データ収集と加工
## 5.2. 自然言語処理
### 5.2.1. 必要なライブラリのインストール
### 5.2.2. 形態素解析
### 5.2.3. Bag of Words (BoW)
### 5.2.4. TF-IDF
### 5.2.5. 極性判定
### 5.2.6. まとめ

In [11]:
# ============================================
# 5.2.1. 必要なライブラリのインストール
# ============================================
# > 参考書籍ではMecabとgensimを用いた処理を実行するようだが
# > 以下の理由より、gensimの使い方のみ学習することとする
# > 理由１：Docker + Mecab の環境構築がかなり面倒そう
# > 理由２：形態素解析処理はJanomeで一通りやったので再度ゼロからやるのは効率が悪い
#          - 参考 : https://github.com/sota0121/slack-msg-analysis
# > 理由３：gensimは使ったことない。Word2Vecはやったことない。
# ** NOTE **
# If Python 3.7.3, pip install gensim --> unable to import 'smart_open.gcs', disabling that module
# 上記の問題を解決するには、smart_openを1.10.0にダウングレードする
# 参考ISSUE：https://github.com/RaRe-Technologies/smart_open/issues/475
# pip install smart_open==1.10.0 を実行すれば良い
from gensim import corpora

In [18]:
# ============================================
# 5.2.3. Bag of Words (BoW)
# - Mecabを使う方法 : skip ** Janomeとsklearn.CountVectorizerでやったことある
# - gensimを使う方法 : to do
# - scikit-learnを使う方法 : skip ** Janomeとsklearn.CountVectorizerでやったことある
# --------------------------
# BoWとは、各単語の出現回数をカウントした情報
# ============================================
words_list = [
    ['子供', 'が', '走る'],
    ['車', 'が', '走る'],
    ['子供', 'の', '脇', 'を', '車', 'が', '走る']
]
# 辞書を作成
word2int_gs = corpora.Dictionary(words_list)
print(word2int_gs)

Dictionary(7 unique tokens: ['が', '子供', '走る', '車', 'の']...)


In [19]:
# 単語と整数の対応
# 各単語にはユニークIDが設定されている
print(word2int_gs.token2id)

{'が': 0, '子供': 1, '走る': 2, '車': 3, 'の': 4, 'を': 5, '脇': 6}


In [20]:
# １番目の文書に含まれる各単語の出現回数を表示
# タプルのリストが出力される
# 各タプルは (単語ID, 出現回数) という構成
print(word2int_gs.doc2bow(words_list[0]))

[(0, 1), (1, 1), (2, 1)]


In [21]:
# 複数文書のBoWを生成する
# Bag of Words を計算し、文書×単語の行列を生成
import numpy as np
from gensim import matutils
bow_gs = np.array(
                    [matutils.corpus2dense(
                        [word2int_gs.doc2bow(words)],
                            num_terms=len(word2int_gs)).T[0]
                                for words in words_list]
            ).astype(np.int)
print(bow_gs)

[[1 1 1 0 0 0 0]
 [1 0 1 1 0 0 0]
 [1 1 1 1 1 1 1]]


In [22]:
# pandas のデータフレームに変換
import pandas as pd
bow_gs_df = pd.DataFrame(bow_gs, columns=list(word2int_gs.values()))
bow_gs_df

,が,子供,走る,車,の,を,脇
0,1,1,1,0,0,0,0
1,1,0,1,1,0,0,0
2,1,1,1,1,1,1,1


In [23]:
# ============================================
# 5.2.4. TF-IDF
# ============================================
# https://github.com/sota0121/slack-msg-analysis/blob/master/src/d031_features/important_word_extraction.py


In [26]:
# ============================================
# 5.2.5. 極性判定
#   - 極性判定とは、文章が肯定的か否定的かを判定すること
#   - 参考 : https://github.com/sugiyamath/sentiment_ja
# ============================================
# 青空文庫から『吾輩は猫である』を取得
import zipfile
import urllib.request
url = 'https://www.aozora.gr.jp/cards/000148/files/789_ruby_5639.zip'
urllib.request.urlretrieve(url, '789_ruby_5639.zip')
with zipfile.ZipFile('789_ruby_5639.zip', 'r') as zipf:
    data = zipf.read('wagahaiwa_nekodearu.txt')  # bytesを変換
text = data.decode('shift_jis')  # shift-jisに変換
# 前処理する
import re
# ルビ　, 注釈, 改行コードを除去
text = re.split(r'\-{5,}', text)[2]
text = text.split('底本: ')[0]
text = re.sub(r'《.+?》', '', text)
text = re.sub(r'［＃.+?］', '', text)
text = text.strip()
# 空白文字を除去
text = text.replace('\u3000', '')
# 改行コードを除去
text = text.replace('\r', '').replace('\n', '')
# 「。」を区切り文字として分割
sentences = text.split('。')
print(sentences[:5])

['一吾輩は猫である', '名前はまだ無い', 'どこで生れたかとんと見当がつかぬ', '何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している', '吾輩はここで始めて人間というものを見た']


In [27]:
# 分かち書きは自作スクリプトを参考にJanomeを利用する
# https://github.com/sota0121/slack-msg-analysis/blob/master/src/d030_processing/morphological_analysis.py


In [32]:
# Text processing : Morphological Analysis
# 1. 形態素解析
# 2. 分かち書き（品詞の指定可能）

from janome.tokenizer import Tokenizer
from tqdm import tqdm

exc_part_of_speech = {
    "名詞": ["非自立", "代名詞", "数"]
}
inc_part_of_speech = {
    "名詞": ["サ変接続", "一般", "固有名詞"],
}


class MorphologicalAnalysis:

    def __init__(self):
        self.janome_tokenizer = Tokenizer()

    def tokenize_janome(self, line: str) -> list:
        # list of janome.tokenizer.Token
        tokens = self.janome_tokenizer.tokenize(line)
        return tokens

    def exists_pos_in_dict(self, pos0: str, pos1: str, pos_dict: dict) -> bool:
        # Retrurn where pos0, pos1 are in pos_dict or not.
        # ** pos = part of speech
        for type0 in pos_dict.keys():
            if pos0 == type0:
                for type1 in pos_dict[type0]:
                    if pos1 == type1:
                        return True
        return False

    def get_wakati_str(self, line: str, exclude_pos: dict,
                       include_pos: dict) -> str:
        '''
        exclude/include_pos is like this
        {"名詞": ["非自立", "代名詞", "数"], "形容詞": ["xxx", "yyy"]}
        '''
        tokens = self.janome_tokenizer.tokenize(line, stream=True)  # 省メモリの為generator
        extracted_words = []
        for token in tokens:
            part_of_speech0 = token.part_of_speech.split(',')[0]
            part_of_speech1 = token.part_of_speech.split(',')[1]
            # check for excluding words
            exists = self.exists_pos_in_dict(part_of_speech0, part_of_speech1,
                                             exclude_pos)
            if exists:
                continue
            # check for including words
            exists = self.exists_pos_in_dict(part_of_speech0, part_of_speech1,
                                             include_pos)
            if not exists:
                continue
            # append(表記揺れを吸収する為 表層形を取得)
            extracted_words.append(token.surface)
        # wakati string with extracted words
        wakati_str = ' '.join(extracted_words)
        return wakati_str


In [34]:
manalyzer = MorphologicalAnalysis()
wakati_msg_list = []
for sentence in tqdm(sentences, desc='wakati> '):
    wakati_sentence = manalyzer.get_wakati_str(sentence, exc_part_of_speech, inc_part_of_speech)
    wakati_msg_list.append(wakati_sentence)
print(wakati_msg_list)

wakati> : 100%|██████████| 7490/7490 [00:42<00:00, 174.30it/s]

['猫', '名前', '見当', 'ニャーニャー いた事 記憶', '人間', 'あと 書生 人間 種族', '書生 話', '考', '掌 スー 感じ', '掌 書生 顔 人間 始', '感じ', '装飾 顔 薬缶', '猫 輪', '顔 真中 突起', '穴 ぷうぷうと 煙', '咽', '人間 煙草', '書生 掌 裏 心持 速力 運転', '書生 自分 眼', '胸', '音 眼 火', '記憶 あと', '気 書生', '兄弟 疋', '母親 姿', '', '眼', '容子', '藁 笹原', '思い 笹原 向う 池', '池', '分別', '書生 迎', 'ニャー ニャー 試み', '池 風 日', '腹', '声', '食物 決心 池 左', '', '我慢 無理やり 人間', '竹垣 穴 邸', '縁 竹垣 路傍 餓死', '一樹 蔭', '垣根 穴 隣家 訪問 通路', '邸 先', '腹 雨 始末 猶予', '方', '家', '書生 人間 機会 遭遇', '', '書生 否や 頸筋 表', '眼 運 天', '我慢', 'おさん 隙 台所', '', '記憶', 'おさん', 'おさん 馬 偸 返報 胸 痞', '最後 家 主人', '下女 主人 宿 猫 御台', '主人 鼻 毛 顔 そん 奥', '主人 口 人', '下女 台所', '家 自分 住', '主人 顔', '職業 教師', '学校 書斎 ぎりほとんど', '家 勉強', '当人 勉強', 'うち 勤勉', '忍び足 書斎 昼寝', '本 涎', '胃弱 皮膚 色 弾力 徴候', '癖 大飯', '大飯 タカジヤスターゼ', '書物', '', '涎 本', '日課', '猫', '教師', '人間 教師', '猫', '主人 教師 友達 かん 不平', '家 主人 人望', '相手 手', '珍重 名前', '主人 傍', '主人 新聞 膝', '昼寝 背中', '主人 別', '経験 飯櫃 炬燵 天気 椽側', '心持 供 寝床 いっしょ', '供 五つ 三つ 一つ 床 一間', '中間 容 余地 運 供 眼 最後', '供 質 猫 猫 声', '例 神経 胃弱 主人 眼 次 部屋', '物 指 尻 ぺたをひどく', '人間 同居 観察 断言', '同衾 供 言語', '

In [35]:
joined_wakati_msgs = ' '.join(wakati_msg_list)
words_all = joined_wakati_msgs.split(' ')
print(words_all)

['猫', '名前', '見当', 'ニャーニャー', 'いた事', '記憶', '人間', 'あと', '書生', '人間', '種族', '書生', '話', '考', '掌', 'スー', '感じ', '掌', '書生', '顔', '人間', '始', '感じ', '装飾', '顔', '薬缶', '猫', '輪', '顔', '真中', '突起', '穴', 'ぷうぷうと', '煙', '咽', '人間', '煙草', '書生', '掌', '裏', '心持', '速力', '運転', '書生', '自分', '眼', '胸', '音', '眼', '火', '記憶', 'あと', '気', '書生', '兄弟', '疋', '母親', '姿', '', '眼', '容子', '藁', '笹原', '思い', '笹原', '向う', '池', '池', '分別', '書生', '迎', 'ニャー', 'ニャー', '試み', '池', '風', '日', '腹', '声', '食物', '決心', '池', '左', '', '我慢', '無理やり', '人間', '竹垣', '穴', '邸', '縁', '竹垣', '路傍', '餓死', '一樹', '蔭', '垣根', '穴', '隣家', '訪問', '通路', '邸', '先', '腹', '雨', '始末', '猶予', '方', '家', '書生', '人間', '機会', '遭遇', '', '書生', '否や', '頸筋', '表', '眼', '運', '天', '我慢', 'おさん', '隙', '台所', '', '記憶', 'おさん', 'おさん', '馬', '偸', '返報', '胸', '痞', '最後', '家', '主人', '下女', '主人', '宿', '猫', '御台', '主人', '鼻', '毛', '顔', 'そん', '奥', '主人', '口', '人', '下女', '台所', '家', '自分', '住', '主人', '顔', '職業', '教師', '学校', '書斎', 'ぎりほとんど', '家', '勉強', '当人', '勉強', 'うち', '勤勉', '忍び足', '書斎', '昼寝', '本', '涎', '胃弱', '皮膚', '色

In [36]:
# 日本語評価極性辞書（東北大学）の取得
urllib.request.urlretrieve('http://www.cl.ecei.tohoku.ac.jp/resources/sent_lex/wago.121808.pn', 'wago.121808.pn')

('wago.121808.pn', <http.client.HTTPMessage at 0x7fc4bf6cf6a0>)

In [37]:
# 極性辞書を読み込む
wago = pd.read_csv('wago.121808.pn', header=None, sep='\t')
wago.head(3)

,0,1
0,ネガ（経験）,あがく
1,ネガ（経験）,あきらめる
2,ネガ（経験）,あきる


In [38]:
# ------------------------------------------------------------
# 極性辞書の極性は4種類あり、ここでは以下のように極性のスコアを定義する
# ポジティブ(score=1)  < ポジ（経験）, ポジ（評価）
# ネガティブ(score=-1) < ネガ（経験）, ネガ（評価）
# ------------------------------------------------------------
# 単語とスコアを対応させる辞書を作成
word2score = {}
values = {'ポジ（経験）': 1, 'ポジ（評価）': 1, 'ネガ（経験）': -1, 'ネガ（評価）': -1}
for word, label in zip(wago.loc[:, 1], wago.loc[:, 0]):
    word2score[word] = values[label]
list(word2score.items())[:3]

[('あがく', -1), ('あきらめる', -1), ('あきる', -1)]

In [44]:
scores = []
for words in wakati_msg_list:
    score = 0
    #  単語が辞書に現れていればスコアを加算
    for word in words:
        if word in word2score:
            score += word2score[word]
    scores.append(score)
df_wordscore = pd.DataFrame({'words': wakati_msg_list, 'score': scores})
df_wordscore

,words,score
0,猫,0
1,名前,0
2,見当,0
3,ニャーニャー いた事 記憶,0
4,人間,0
...,...,...
7485,,0
7486,底本 夏目 漱石 全集 ちく 文庫 筑摩書房 昭和 月 発行 底本 本 筑摩 全集 類聚 夏...,0
7487,入力 柴田 卓治 校正 渡部 峰子 のし 田尻 幹 高橋 真 也 しず 瀬戸 さえ子 月 公...,0
7488,入力 校正 制作 ボランティア 皆さん,0
